In [12]:
import json
import random

random.seed(42)
# 读取原始JSON文件
data = []

with open('../data/angular_filtered/subsets/generation/test_race.json', 'r', encoding='utf-8') as f:
    race_data = json.load(f)
with open('../data/angular_filtered/subsets/generation/test_gpt35_zeroshot.json', 'r', encoding='utf-8') as f:
    zeroshot_data = json.load(f)
with open('../data/angular_filtered/subsets/generation/test_gpt35_rag.json', 'r', encoding='utf-8') as f:
    rag_data = json.load(f)
with open('../data/angular_filtered/subsets/generation/test_gpt35_model_classified_rag.json', 'r', encoding='utf-8') as f:
    model_rag_data = json.load(f)
with open('../data/angular_filtered/subsets/generation/test_nngen.json', 'r', encoding='utf-8') as f:
    nngen_data = json.load(f)
    

In [13]:
for index in range(len(rag_data)):
    item = {
        'diff': race_data[index]['diff'],
        'human': race_data[index]['msg'],
        'race': race_data[index]['race'],
        'zeroshot': zeroshot_data[index]['chatgpt_zeroshot'],
        'rag': rag_data[index]['chatgpt_rag'],
        'classified_rag': model_rag_data[index]['chatgpt_rag'],
        'nngen': nngen_data[index]['nngen'],
        'type': race_data[index]['type']
    }
    data.append(item)

with open('../data/angular_filtered/subsets/generation/test_all.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [19]:

# 确保JSON数据是一个列表
if not isinstance(data, list):
    raise ValueError("JSON数据不是一个列表")

# 随机选择100个项目
sample_size = 100
if len(data) < sample_size:
    raise ValueError("数据量少于100个项目")

initial_sample = random.sample(data, sample_size)

# 从剩余的项目中再随机选择100个项目
remaining_data = [item for item in data if item not in initial_sample]
second_sample = random.sample(remaining_data, sample_size)

# 将选定的项目写入一个新的JSON文件
with open('../data/angular_filtered/subsets/human_eval/no_human_ref_test.json', 'w', encoding='utf-8') as f:
    json.dump(initial_sample, f, ensure_ascii=False, indent=4)
    
# 将选定的项目写入一个新的JSON文件
with open('../data/angular_filtered/subsets/human_eval/human_ref_test.json', 'w', encoding='utf-8') as f:
    json.dump(second_sample, f, ensure_ascii=False, indent=4)

print("随机提取的100个项目已保存到output.json文件中。")


随机提取的100个项目已保存到output.json文件中。


In [20]:
## 统计随机提取的100个项目的类型分布
types = {}
for item in initial_sample:
    if item['type'] not in types:
        types[item['type']] = 0
    types[item['type']] += 1

print("类型分布：")
for key, value in types.items():
    print(f"{key}: {value}")

类型分布：
refactor: 15
docs: 18
fix: 18
test: 9
feat: 19
chore: 15
ci: 1
style: 2
perf: 2
build: 1


In [21]:
## 统计随机提取的100个项目的类型分布
types = {}
for item in second_sample:
    if item['type'] not in types:
        types[item['type']] = 0
    types[item['type']] += 1

print("类型分布：")
for key, value in types.items():
    print(f"{key}: {value}")

类型分布：
feat: 19
fix: 17
docs: 20
build: 3
test: 7
chore: 19
refactor: 11
ci: 2
style: 1
perf: 1
